<a href="https://colab.research.google.com/github/Johanhms/Learning_Python/blob/main/ETL_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Instala la librería SQLAlchemy para conectar con bases de datos
!pip install --quiet sqlalchemy

import pandas as pd
import io

# Simula la extracción de datos de origen
# En un escenario real, cargarías un archivo con pd.read_csv()
# o te conectarías a una base de datos con SQLAlchemy.
data_origen = """id_venta,fecha_venta,id_producto,cantidad,precio_unitario,region
1,2023-01-05,101,5 unidades,25.50,norte
2,2023/01/06,102,10 un.,15.00,sur
3,2023-01-07,103,3,50.00,Norte
"""

df_raw = pd.read_csv(io.StringIO(data_origen))

print("DataFrame de origen:")
print(df_raw)

DataFrame de origen:
   id_venta fecha_venta  id_producto    cantidad  precio_unitario region
0         1  2023-01-05          101  5 unidades             25.5  norte
1         2  2023/01/06          102      10 un.             15.0    sur
2         3  2023-01-07          103           3             50.0  Norte


In [6]:
# Copia el DataFrame para no modificar el original
df_limpio = df_raw.copy()

# 1. Limpia y convierte la columna 'cantidad' a un tipo numérico
df_limpio['cantidad'] = df_limpio['cantidad'].str.replace(r'[^0-9]', '', regex=True).astype(int)

# 2. Convierte la columna 'fecha_venta' a tipo datetime
df_limpio['fecha_venta'] = pd.to_datetime(df_limpio['fecha_venta'], errors='coerce', format='mixed')

# 3. Normaliza la columna 'region' a mayúsculas y minúsculas
df_limpio['region_normalizada'] = df_limpio['region'].str.capitalize()

# 4. Calcula el 'precio_total'
df_limpio['precio_total'] = df_limpio['cantidad'] * df_limpio['precio_unitario']

# Selecciona y reordena las columnas finales
df_final = df_limpio[['id_venta', 'fecha_venta', 'id_producto', 'cantidad', 'precio_total', 'region_normalizada']]

print("\nDataFrame final (transformado):")
print(df_final)


DataFrame final (transformado):
   id_venta fecha_venta  id_producto  cantidad  precio_total  \
0         1  2023-01-05          101         5         127.5   
1         2  2023-01-06          102        10         150.0   
2         3  2023-01-07          103         3         150.0   

  region_normalizada  
0              Norte  
1                Sur  
2              Norte  


In [7]:
# Opción 1: Guardar el DataFrame como un archivo CSV
df_final.to_csv('ventas_limpias.csv', index=False)
print("\nDatos cargados exitosamente a 'ventas_limpias.csv'")

# Opción 2: Cargar el DataFrame a una base de datos SQLite (ideal para practicar)
from sqlalchemy import create_engine

# Crea una conexión a una base de datos en memoria (SQLite)
engine = create_engine('sqlite:///:memory:')

# Carga el DataFrame en una tabla SQL
df_final.to_sql('ventas_finales', con=engine, index=False, if_exists='replace')

print("\nDatos cargados a la base de datos 'ventas_finales'.")

# Verifícalo consultando la tabla
with engine.connect() as conn:
    df_verificacion = pd.read_sql("SELECT * FROM ventas_finales", conn)
    print("\nVerificación de la carga desde la base de datos:")
    print(df_verificacion)


Datos cargados exitosamente a 'ventas_limpias.csv'

Datos cargados a la base de datos 'ventas_finales'.

Verificación de la carga desde la base de datos:
   id_venta                 fecha_venta  id_producto  cantidad  precio_total  \
0         1  2023-01-05 00:00:00.000000          101         5         127.5   
1         2  2023-01-06 00:00:00.000000          102        10         150.0   
2         3  2023-01-07 00:00:00.000000          103         3         150.0   

  region_normalizada  
0              Norte  
1                Sur  
2              Norte  
